In [1]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier, export_text
import pandas as pd
import random
import seaborn as sns

In [2]:
naives_bayes=MultinomialNB()
decision_tree=DecisionTreeClassifier()
folds=[2,4,6,8,10,12,14,16]
splits=[0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55]
seed=6896#random.randint(0, 10000)
data="adult.data"
df = pd.read_csv(data)
#df.info()
#df.describe()
#print(df.head())
#print(df[['<=50K']].value_counts())
df.head()
# a = df.iloc[:,14]
# count=0
# count1=0

# for i in a:
#     if i==" <=50K":
#         count1+=1
#     #if i=='<=50K':
#     count+=1

# print(count-(count1))


,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
0,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
1,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
2,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
3,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
4,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K


In [3]:
replace_dict={}
for index,j in enumerate(df):
    count =0
    for i in set(df.iloc[:, index]):
        if(type(i)==str):
            replace_dict[i]=count
            count+=1
print(replace_dict)

{' Without-pay': 0, ' Never-worked': 1, ' State-gov': 2, ' Self-emp-not-inc': 3, ' Self-emp-inc': 4, ' Local-gov': 5, ' Private': 6, ' ?': 32, ' Federal-gov': 8, ' 10th': 0, ' Doctorate': 1, ' 11th': 2, ' Assoc-voc': 3, ' Preschool': 4, ' Assoc-acdm': 5, ' 5th-6th': 6, ' Prof-school': 7, ' 9th': 8, ' Some-college': 9, ' 7th-8th': 10, ' 1st-4th': 11, ' 12th': 12, ' HS-grad': 13, ' Bachelors': 14, ' Masters': 15, ' Married-civ-spouse': 0, ' Married-spouse-absent': 1, ' Never-married': 2, ' Divorced': 3, ' Widowed': 4, ' Separated': 5, ' Married-AF-spouse': 6, ' Transport-moving': 0, ' Craft-repair': 1, ' Machine-op-inspct': 2, ' Other-service': 3, ' Tech-support': 4, ' Armed-Forces': 5, ' Handlers-cleaners': 7, ' Priv-house-serv': 8, ' Farming-fishing': 9, ' Sales': 10, ' Protective-serv': 11, ' Prof-specialty': 12, ' Exec-managerial': 13, ' Adm-clerical': 14, ' Not-in-family': 0, ' Own-child': 1, ' Wife': 2, ' Unmarried': 3, ' Husband': 4, ' Other-relative': 5, ' White': 0, ' Black': 1,

Load Dataset

In [4]:
classes = df.iloc[:, -1]
numeric_df = df.iloc[:,:-1].replace(replace_dict)
columns = df.columns.array[:-1]
class_values = set(classes)
numeric_df.head()

,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
0,50,3,83311,14,13,0,13,4,0,0,0,0,13,2
1,38,6,215646,13,9,3,7,0,0,0,0,0,40,2
2,53,6,234721,2,7,0,7,4,1,0,0,0,40,2
3,28,6,338409,14,13,0,12,2,1,1,0,0,40,5
4,37,6,284582,15,14,0,13,2,0,1,0,0,40,2


In [5]:

def classification_report_csv(report,mydata):
    lines = report.split('\n')
    for line in lines[2:-3]:
        row = {}
        row_data = line.split('      ')
        if len(row_data)==6:
            row['class'] = row_data[1]
            row['precision'] = row_data[2]
            row['recall'] = row_data[3]
            row['f1_score'] = row_data[4]
            row['support'] = row_data[5]
            mydata.append(row)

In [6]:
def print_model_evaluation(model, test_data, test_classes, class_values):
    predictions = model.predict(test_data)
    accuracy = accuracy_score(test_classes, predictions)
    print("Accuracy score: ",accuracy)
    confusion_mat = confusion_matrix(test_classes, predictions)
    print(confusion_mat)
    report = classification_report(test_classes, predictions, target_names=class_values, zero_division=0)
    print(report)
    return report


Split data

In [7]:
test_id = 1
decision_report=[]
naives_report=[]
for split_ratio in splits:
    (training_data, test_data, training_classes, test_classes) = train_test_split(
        numeric_df,
        classes,
        test_size=split_ratio,
        random_state=seed
    )
    print(f"Prueba {test_id}")
    print(f"Training data: {(1 - split_ratio) * 100}    Test data: {(split_ratio * 100)}")
    naive_bayes_model = naives_bayes.fit(training_data, training_classes)
    decision_tree_model = decision_tree.fit(training_data, training_classes) 
    print("Decision tree report")
    tree_text = export_text(decision_tree_model, feature_names=columns)
    #print(tree_text)
    decision_report1 = print_model_evaluation(decision_tree_model, test_data, test_classes, class_values)
    decision_report2=classification_report_csv(decision_report1,decision_report)
    print("Naive Bayes report")
    naives_report1=print_model_evaluation(naive_bayes_model, test_data, test_classes, class_values)
    naives_report2=classification_report_csv(naives_report1,naives_report)

    test_id += 1
dataframe = pd.DataFrame.from_dict(naives_report)
dataframe.to_csv('naives_report.csv', index = False)
dataframe = pd.DataFrame.from_dict(decision_report)
dataframe.to_csv('decision_report.csv', index = False)

Prueba 1
Training data: 80.0    Test data: 20.0
Decision tree report
Accuracy score:  0.8138820638820639
[[4299  639]
 [ 573 1001]]
              precision    recall  f1-score   support

        >50K       0.88      0.87      0.88      4938
       <=50K       0.61      0.64      0.62      1574

    accuracy                           0.81      6512
   macro avg       0.75      0.75      0.75      6512
weighted avg       0.82      0.81      0.82      6512

Naive Bayes report
Accuracy score:  0.785012285012285
[[4723  215]
 [1185  389]]
              precision    recall  f1-score   support

        >50K       0.80      0.96      0.87      4938
       <=50K       0.64      0.25      0.36      1574

    accuracy                           0.79      6512
   macro avg       0.72      0.60      0.61      6512
weighted avg       0.76      0.79      0.75      6512

Prueba 2
Training data: 75.0    Test data: 25.0
Decision tree report
Accuracy score:  0.8115479115479115
[[5333  827]
 [ 707 1273]]
 

Validate

In [8]:
def get_fold_splitted_data(data, classes, fold):
    skf = StratifiedKFold(fold, shuffle=False)
    for train_index, test_index in skf.split(data, classes):
        training_data, test_data = data.iloc[train_index], data.iloc[test_index]
        training_classes, test_classes = classes[train_index], classes[test_index]
    return (training_data, test_data, training_classes, test_classes)


In [9]:
test_id = 1
decision_report=[]
naives_report=[]
for fold in folds:
    (training_data, test_data, training_classes, test_classes) = get_fold_splitted_data(numeric_df, classes, fold)
    print(f"Pruebas {test_id} ")
    print("Folds: " + str(fold))
    naive_bayes_model = naives_bayes.fit(training_data, training_classes)
    decision_tree_model = decision_tree.fit(training_data, training_classes) 
    print("Decision tree report")
    tree_text = export_text(decision_tree_model, feature_names=columns)
    #print(tree_text)
    decision_report1 = print_model_evaluation(decision_tree_model, test_data, test_classes, class_values)
    decision_report2=classification_report_csv(decision_report1,decision_report)
    print("Naive Bayes report")
    naives_report1=print_model_evaluation(naive_bayes_model, test_data, test_classes, class_values)
    naives_report2=classification_report_csv(naives_report1,naives_report)
    test_id += 1
dataframe = pd.DataFrame.from_dict(naives_report)
dataframe.to_csv('naives_report1.csv', index = False)
dataframe = pd.DataFrame.from_dict(decision_report)
dataframe.to_csv('decision_report1.csv', index = False)

Pruebas 1 
Folds: 2
Decision tree report
Accuracy score:  0.8135749385749386
[[10798  1561]
 [ 1474  2447]]
              precision    recall  f1-score   support

        >50K       0.88      0.87      0.88     12359
       <=50K       0.61      0.62      0.62      3921

    accuracy                           0.81     16280
   macro avg       0.75      0.75      0.75     16280
weighted avg       0.82      0.81      0.81     16280

Naive Bayes report
Accuracy score:  0.7843366093366093
[[11811   548]
 [ 2963   958]]
              precision    recall  f1-score   support

        >50K       0.80      0.96      0.87     12359
       <=50K       0.64      0.24      0.35      3921

    accuracy                           0.78     16280
   macro avg       0.72      0.60      0.61     16280
weighted avg       0.76      0.78      0.75     16280

Pruebas 2 
Folds: 4
Decision tree report
Accuracy score:  0.8141277641277641
[[5404  775]
 [ 738 1223]]
              precision    recall  f1-score   su

In [10]:
# calculate the correlations
correlations = numeric_df.corr()

# plot the heatmap 
sns.heatmap(correlations, xticklabels=correlations.columns, yticklabels=classes, annot=True)

# plot the clustermap 
sns.clustermap(correlations, xticklabels=correlations.columns, yticklabels=classes, annot=True)